In [2]:
import paramiko
import time
import sys

ip = "172.16.16.25"
user = "root"
pw = "asd123!@"

def cc(command):
    """SSH로 명령을 실행하고 결과를 출력"""
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(command)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    if err:
        print("⚠️ 오류 발생:", err)
    return out.strip()

##################### server #####################
def dhcp_install():
    cc("dnf install -y dhcp-server")

def set_dhcp_file():

    
    # 사용자 입력
    subnet = input("위에서 확인한 subnet을 입력하세요 ex. 172.16.0.0: ")
    netmask = input("위에서 확인한 netmask을 입력하세요 ex. 255.255.0.0: ")
    exclusion_start = input("제외할 ip 시작 주소를 적으세요 ex. 172.16.16.100: ")
    exclusion_end = input("제외할 ip 마지막 주소를 적으세요 ex. 172.16.16.200: ")
    gateway = input("게이트웨이를 입력하세요 ex. 172.16.0.1: ")
    domain = input("도메인 서버를 입력하세요 ex. 8.8.8.8: ")
    pool = input("pool의 이름을 지정하세요 ex. 'example.local': ")


    
    # /etc/dhcp/dhcpd.conf
    cmd = f"""
echo '
default-lease-time 600;
max-lease-time 7200;
authoritative;

subnet {subnet} netmask {netmask} {{
    range {exclusion_start} {exclusion_end};
    option routers {gateway};
    option subnet-mask {netmask};
    option domain-name-servers {domain}, 8.8.8.8;
    option domain-name "{pool}";
}}
' > /etc/dhcp/dhcpd.conf
"""
    cc(cmd)
    print("/etc/dhcp/dhcpd.conf 파일 업데이트 완료")
##################### client #####################
def dhcp_restart():
        
    cc("systemctl enable --now dhcpd")
    print(cc("systemctl status dhcpd"))

###################### 실행 ########################
def dhcp_request():
    cc("sudo dnf install dhcp -y")
    cc("sudo dnf install dhcp-server -y")
    

def dhcp_server():
    dhcp_install()
    set_dhcp_file()
    dhcp_restart()

def dhcp_client():
    dhcp_request()

#dhcp_server()
dhcp_client()



1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: ens160: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 00:0c:29:f3:5b:f3 brd ff:ff:ff:ff:ff:ff
    altname enp3s0
    inet 172.16.16.25/16 brd 172.16.255.255 scope global noprefixroute ens160
       valid_lft forever preferred_lft forever
    inet6 fe80::20c:29ff:fef3:5bf3/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft foreve

위에서 확인한 subnet을 입력하세요 ex. 172.16.0.0:  172.16.0.0
위에서 확인한 netmask을 입력하세요 ex. 255.255.0.0:  255.255.0.0
제외할 ip 시작 주소를 적으세요 ex. 172.16.16.100:  172.16.16.100
제외할 ip 마지막 주소를 적으세요 ex. 172.16.16.200:  172.16.16.200
게이트웨이를 입력하세요 ex. 172.16.0.1:  172.16.0.1
도메인 서버를 입력하세요 ex. 8.8.8.8:  8.8.8.8
pool의 이름을 지정하세요 ex. 'example.local':  example.com


/etc/dhcp/dhcpd.conf 파일 업데이트 완료
● dhcpd.service - DHCPv4 Server Daemon
     Loaded: loaded (/usr/lib/systemd/system/dhcpd.service; enabled; preset: disabled)
     Active: active (running) since Mon 2025-11-03 13:32:08 KST; 2min 19s ago
       Docs: man:dhcpd(8)
             man:dhcpd.conf(5)
   Main PID: 4679 (dhcpd)
     Status: "Dispatching packets..."
      Tasks: 1 (limit: 10857)
     Memory: 4.6M
        CPU: 10ms
     CGroup: /system.slice/dhcpd.service
             └─4679 /usr/sbin/dhcpd -f -cf /etc/dhcp/dhcpd.conf -user dhcpd -group dhcpd --no-pid ens160

Nov 03 13:32:08 localhost.localdomain dhcpd[4679]: Config file: /etc/dhcp/dhcpd.conf
Nov 03 13:32:08 localhost.localdomain dhcpd[4679]: Database file: /var/lib/dhcpd/dhcpd.leases
Nov 03 13:32:08 localhost.localdomain dhcpd[4679]: PID file: /var/run/dhcpd.pid
Nov 03 13:32:08 localhost.localdomain dhcpd[4679]: Source compiled to use binary-leases
Nov 03 13:32:08 localhost.localdomain dhcpd[4679]: Wrote 0 leases to leases file.
N